In [6]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
import time
import numpy as np
from PIL import Image
import os

In [7]:

def load_model(model_path):
    """Load the trained MobileNet V2 model"""
    model = models.mobilenet_v2(pretrained=False)
    model.classifier[1] = nn.Linear(model.classifier[1].in_features, 4)
    
    checkpoint = torch.load(model_path, map_location='cpu')
    model.load_state_dict(checkpoint)
    model.eval()
    return model


def preprocess_image(image_path):
    """Preprocess image for inference"""
    transform = transforms.Compose([
        transforms.Resize((224, 224)),
        transforms.ToTensor(),
        transforms.Normalize(mean=[0.5], 
                           std=[0.5])
    ])
    
    image = Image.open(image_path).convert('RGB')
    image = transform(image)
    return image.unsqueeze(0) 

In [8]:
def measure_inference_time(model, input_tensor, num_runs=100):
    """Measure inference time over multiple runs"""
    model.eval()
    
    # Warm up the model
    with torch.no_grad():
        for _ in range(10):
            _ = model(input_tensor)
    
    # Measure inference time
    times = []
    with torch.no_grad():
        for _ in range(num_runs):
            start_time = time.time()
            output = model(input_tensor)
            end_time = time.time()
            times.append(end_time - start_time)
    
    return times



In [9]:
import torch
import torch.nn as nn
import torchvision.transforms as transforms
from torchvision import models
import time
import numpy as np
from PIL import Image
import os

# Model path
model_path = 'outputs/checkpoints/mobilenet_v2_epoch_29.pth'

# Check if model exists
if not os.path.exists(model_path):
    print(f"Model not found at {model_path}")
    exit()

# Load the trained model
print("Loading model...")
model = models.mobilenet_v2(pretrained=False)
# Modify the final layer for your classification task (assuming 4 classes)
model.classifier[1] = nn.Linear(model.classifier[1].in_features, 4)

# Load the trained weights
checkpoint = torch.load(model_path, map_location='cpu')
model.load_state_dict(checkpoint)
model.eval()
print("Model loaded successfully!")

# Create a dummy input tensor for testing
dummy_input = torch.randn(1, 3, 224, 224)

# Warm up the model
print("Warming up model...")
with torch.no_grad():
    for _ in range(10):
        _ = model(dummy_input)

# Measure inference time
print("Measuring inference time...")
times = []
with torch.no_grad():
    for _ in range(100):
        start_time = time.time()
        output = model(dummy_input)
        end_time = time.time()
        times.append(end_time - start_time)

# Calculate statistics
mean_time = np.mean(times)
std_time = np.std(times)
min_time = np.min(times)
max_time = np.max(times)

print(f"\nInference Time Statistics:")
print(f"Mean: {mean_time:.4f} seconds")
print(f"Std: {std_time:.4f} seconds")
print(f"Min: {min_time:.4f} seconds")
print(f"Max: {max_time:.4f} seconds")
print(f"FPS: {1/mean_time:.2f}")

Loading model...
Model loaded successfully!
Warming up model...


/media/ayan/Ayan/conda-ins/envs/myenv/lib/python3.9/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/media/ayan/Ayan/conda-ins/envs/myenv/lib/python3.9/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=None`.
  warnings.warn(msg)
/tmp/ipykernel_423396/220357931.py:25: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_on

Measuring inference time...

Inference Time Statistics:
Mean: 0.0165 seconds
Std: 0.0022 seconds
Min: 0.0115 seconds
Max: 0.0227 seconds
FPS: 60.71
